# Projekt zur Spurerkennung im Wahlfach "Digitale Bildverarbeitung" 

In diesem Jupyter-Notebook wird eine Fahrbahnmarkierungserkennung implementiert. Anhand von Bildern und Videos von Udacity und KITTI wird diese Erkennung auf verschiedenste Weise getestet.

Um eine Fahrbahn erkennen zu können müssen folgende Schritte während des Programmablaufs abgearbeitet werden:

    - Kamerakalibrierung
    - Perspektivtransformation
    - Maskieren des Bildes mit gelben und weißen Farbmasken
    - "Sliding Windows"
    - Berechnung der Polynome für die Eingrenzung der Fahrbahn
    - Rücktransformation der berechneten Punkte der Polynome
    - Einzeichnen der Fläche zwischen den Polynomen

Im folgenden werden die jeweils verwendeten Funktionen genauer erklärt, am Ende finden sich die Bilder zur Kamerakalibrierung sowie zur Spurerkennung.

<sup>Bearbeitet wurde das Projekt von Alexander Schulte(), Edmund Krain () und Marcel Fleck (9611872)</sup>

In [ ]:
# imports
import cv2 as cv
import numpy as np
import time
import glob
from prettytable import PrettyTable
from deprecated import deprecated

# Wichtige Attribute/Flags

Hier kann gewählt werden, welches Video für die Spurerkennung verwendet werden soll. Zur Auswahl stehen das "project"-Video sowie das "challenge"-Video.

Die Flag ```DEBUG_MODE``` dient zur Aktivierung des von uns implementierten Debug-Modus. Dabei werden mehrere Fenster mit unterschiedlichen Masken und Einstellungsmöglichkeiten angezeigt, um die optimalen Einstellungen durch ausprobieren herausfinden zu können. Zu den jeweiligen Debug-Optionen folgt in den jeweiligen Abschnitten mehr. Da mit der Aktivierung des Debug-Modus das Video nicht mehr automatisch läuft sondern manuell einzelne Frames durchlaufen werden können, ist der Debug-Modus standardweise ausgeschalten.

In [ ]:
# video_file = "project"
video_file = "challenge"

DEBUG_MODE = False

# Zeitmessungen

Diese Funktionen dienen der Bestimmung von zeitaufwändigen Arbeitsschritten, um eine Verbesserung der Framerate zu erleichtern.

Bei jedem Aufruf der ```start_time_measurement```-Funktion wird in einem Array ein Objekt mit den Eigenschaften "name", "start", "end" und "count" erstellt.
Der Name dient als hierbei als eindeutige Identifizierung der Objekte.

Nach der Erstellung des Objekts wird die aktuelle Zeit nanosekundengenau gespeichert. Da diese Zeit so spät wie nur möglich abgespeichert wird, sind die Messergebnisse nicht von den Operationen der Funktion beeinträchtigt.
Die gespeicherte Zeit wird nun in das "start"-Attribut des Objektes hinzugefügt. Sie wird für die spätere Berechnung der Dauer benötigt.

Bei einem Aufruf von ```end_time_measurement``` wird zuallererst die aktuelle Zeit, wieder auf Nanosekunden genau, abgespeichert. Hier wird die Zeit als erster Arbeitsschritt gespeichert, um wieder die Messergebnisse nicht durch die weiteren Operationen der Funktion zu beeinträchtigen. Diese Zeit wird dann im "end"-Attribut des Objekts gespeichert.

Falls ```start_time_management``` mit einem Namen aufgerufen wird, für den es bereits einen Eintrag im Arry gibt, wird eine Variable hochgezählt. Mit Hilfe dieser lässt sich später die durchschnittliche Dauer der Operationen berechnen.

Beide Funktionen beinhalten Abfragen, die ein doppeltes Starten oder auch ein Beenden vor einem Start verhindern.

```analyse_time_measurement``` stellt die gesammelten Daten in einer Tabelle an.
Hierfür werden erst die durchschnittliche, die minimale und die maximale Dauer der jeweiligen, per Namen definierten Objekte berechnet und von Nanosekunden zu Millisekunden umgerechnet.
Auch die Anzahl der Aufrufe wird in der Tabelle angezeigt.

Abbildung 1 zeigt ein Beispiel für eine solche Tabelle:

<p align="center">
  <img src="img/documentation/timemeasurement.png" height="35%" width="35%" />
  <br>
  <em>Abbildung 1: Tabelle zur Darstellung der Zeitmessungsergebnisse</em>
</p>

Im Allgemeinen werden diese Funktionen jeweils vor und nach wichtigen Arbeitsschritten in der Spurerkennung (wie z.B. das Maskieren der Frames) aufgerufen, um feststellen zu können, ob die jeweiligen Schritte verbessert werden müssen.

In [ ]:
time_measurements = {}

def start_time_measurement(eventName):
    """
    Use this function to start a time measurement for a specific event.
    A end_time_measurement() call with the same name must be called before the next start_time_measurement() call.
    """
    add = False
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [],
            "end": [],
            "count": 0
        }
        add = True
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       add = True
    
    #start measurement as late as possible
    if add:
        time_measurements[eventName]["start"].append(time.perf_counter_ns())

def end_time_measurement(eventName):
    """
    Use this function to end a time measurement for a specific event.
    A time measurement with the same name must be started with start_time_measurement() before it can be ended.
    """
    #end measurement as fast as possible
    temp_time = time.perf_counter_ns()
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
        time_measurements[eventName]["end"].append(temp_time)
        time_measurements[eventName]["count"] += 1
    
def analyse_time_measurements():
    """
    Analyse time measurements and print them in a table.
    Multiple events with the same name are averaged.
    """
    time_measurements_table = PrettyTable(["Name", "Avg. [ms]", "Min. [ms]", "Max. [ms]", "Occurrences [compl.]"])
    time_measurements_table.align["Name"] = "l"
    time_measurements_table.align["Avg. [ms]"] = "r"
    time_measurements_table.align["Min. [ms]"] = "r"
    time_measurements_table.align["Max. [ms]"] = "r"
    time_measurements_table.align["Occurrences [compl.]"] = "r"
    for key, event in time_measurements.items():
        timings = []
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
            continue
        
        for i in range(len(event["start"])):
            timing = (event["end"][i] - event["start"][i])
            #exclude 0 values
            if timing >= 0:
                timing = timing / (1000 * 1000) #convert from ns to ms
                timings.append(timing)

        event["min"] = min(timings) 
        event["max"] = max(timings)
        event["avg"] = sum(timings) / len(event["start"])

        time_measurements_table.add_row([key, '{0:.2f}'.format(event["avg"]), '{0:.2f}'.format(event["min"]), '{0:.2f}'.format(event["max"]), event["count"]])
        
    print(time_measurements_table)

# Kamerakalibrierung

Die Abbildung eines 3D-Objekts auf eine 2D-Bildebene mit einer Kamera wird durch die internen Größen einer Kamera beeinflusst. Dazu gehören zum Beispiel die Bildmitte, die Brennweite und auch Kameraverzerrungsparameter.

Mit Hilfe der Kamerakalibrierung werden die sogenannten "intrinsische" und "extrinsische" Parameter berechnet. Intrinsische Parameter beschreiben die Kalibriermatrix (Kamera zu Pixel), während die extrinsischen Parameter die Beziehung zwischen dem Koordinatensystem der Kamera und dem Welt-Koordinatensystem beschreiben (Welt zu Kamera).

Zusammen bilden diese Parameter eine Projektionsmatrix, die es ermöglicht, gemachte Bilder im Welt-Koordinatensystem darzustellen (die Bilder werden entzerrt). 

In folgendem Abschnitt wird die Kamera mit Hilfe von Schachbrett-Bildern kalibriert. Diese Aufgabe mit den Ergebnissen auf den Schachbrettbildern befinden sich im JupyterNotebook [Projekt_Spurerkennung_v2.jpynb](Projekt_Spurerkennung_v2.ipynb).


Mit den unten definierten Funktionen (siehe ```undistort_image``` und ```undistort_image_remap```) werden dann die Frames der Videos anhand der berechneten Transformations-Matrix entzerrt, um eine tatsächliche Darstellung im Welt-Koordinatensystem zu erhalten. Es wurden zwei Funktionen implementiert:

- ```undistort_image```, wie sie in [Projekt_Spurerkennung_v2.jpynb](Projekt_Spurerkennung_v2.ipynb) gezeigt wurde, berechnet für jedes mitgegebene Frame die passenden Parameter und entzerrt das Bild dementsprechend.
- ```undistort_image_remap``` berechnet eine Matrix anhand des ersten übergebenen Frames und wendet diese Matrix bei weiteren Aufrufen auf die jeweils übergebenen Frames an.

### <p style = "color: red"> Was genau machen die Funktionen jeweils -> Wir können eig nicht 50% schneller sein</p>
### <p style = "color: red"> Müssen wir nochmal Benchmarken, die Funktionen machen genau das gleiche, siehe folgende Links</p>
### <p style = "color: red"> undistort_image: https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga69f2545a8b62a6b0fc2ee060dc30559d</p>
### <p style = "color: red"> undistort_image_remap: https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#ga7dfb72c9cf9780a347fbe3d1c47e5d5a</p>

Das führt dazu, dass mit ```undistort_image_remap``` eine ungefähr 75% performantere Lösung für die Kamerakalibrierung gefunden wurde (vgl. Abbildung 2).

<p align="center">
  <img src="img/documentation/calibrationFaster.png" width="15%" height="15%" />
  <br>
  <em>Abbildung 3: Dauer jeder Funktion für jeweils 3 Durchläufe</em>
</p>

In [ ]:
# size of chessboard, minimum error with (7, 6), but there were severe artifacts at the borders (see error calculation at the end)
chessboard_x, chessboard_y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
obj_points = np.zeros((chessboard_y * chessboard_x, 3), np.float32)
obj_points[:, :2] = np.mgrid[0:chessboard_x, 0:chessboard_y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
object_points = []  # 3d point in real world space
image_points = []  # 2d points in image plane.

# use all calibration images
images = glob.glob("./img/Udacity/calib/*.jpg")
for i, frame_name in enumerate(images):
    image = cv.imread(frame_name)
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(image_gray, (chessboard_x, chessboard_y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        object_points.append(obj_points)
        corners2 = cv.cornerSubPix(
            image_gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        image_points.append(corners)

ret, camera_matrix, dist_coefficient, rvecs, tvecs = cv.calibrateCamera(
    object_points, image_points, image_gray.shape[::-1], None, None
)

image_height, image_width = image.shape[:2]
new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(camera_matrix, dist_coefficient, (image_width, image_height), 1, (image_width, image_height))

map_x, map_y = cv.initUndistortRectifyMap(camera_matrix, dist_coefficient, None, new_camera_matrix, (image_width, image_height), 5)

# ------- define functions for image processing -------
@deprecated
def undistort_image(img):
    img_undistorted = cv.undistort(img, camera_matrix, dist_coefficient, None, new_camera_matrix)
    # crop the image
    x, y, image_width, image_height = roi
    return img_undistorted[y : y + image_height, x : x + image_width]

# ~75% faster than undistort_image()
def undistort_image_remap(img):
    """Undistort image using remap function without recalculating the map every time."""
    dst = cv.remap(img, map_x, map_y, cv.INTER_LINEAR)
    # crop the image
    x, y, image_width, image_height = roi
    return dst[y : y + image_height, x : x + image_width]

# Perspektivtransformation

Die Perspektivtransformation wird verwendet, um die "Region of Interest" festzulegen. Dadurch muss nicht mehr das gesamte Bild verarbeitet werden, sondern es kann nur ein der Bereich, in dem sich die Fahrspur befindet, zur Verarbeitung genutzt werden.

Hierbei werden vier feste Punkte im originalen Bild ausgewählt. Diese Punkte werden dann in einem Bild mit den selben Dimensionen wie das Originalbild dargestellt, um eine rechteckige Beziehung zwischen den Punkten herzustellen (siehe Abbildung 3). Die Transformation geschieht mit der Funktion ```warp_image_udacity```.

<p align="center">
  <img src="img/documentation/perspectiveTransformation.png" />
  <br>
  <em>Abbildung 3: Beispiel für implementierte Perspektivtransformation</em>
</p>

Um die mit einer später erklärten Funktion (```sliding_windows```) in dem transformierten Bild gefundenen Punkte in dem originalen Bild anzeigen zu können, müssen diese Punkte zurücktransformiert werden. Um Rechenleistung zu sparen, werden tatsächlich nur die Punkte transformiert (siehe ```rewarp_points_udacity```), bei einem ersten Ansatz wurde das gesamte Bild zurücktransformiert (siehe ```rewarp_image_udacity```). 

Eine weitere Überlegung war es, das Bild nach der Perspektivtransformation zu verkleinern. Damit würde die Anzahl an zu verarbeitenden Pixeln deutlich sinken, was eine Verbesserung der FPS mit sich bringen würde. Jedoch traten Probleme bei der Rücktransformation der Punkte des kleinen Bildes in die des Originalframes auf, weshalb diese Idee nicht weiter verfolgt wurde.

In [ ]:
# udacity images
src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])

M_warp = cv.getPerspectiveTransform(src_udacity, dst_udacity)
M_rewarp = cv.getPerspectiveTransform(dst_udacity, src_udacity)


def warp_image_udacity(img):
    """Warp image to bird's eye view. Uses fix transformation matrix.

    Args:
        img (_type_): input image

    Returns:
        _type_: image in bird's eye view
    """
    image = cv.warpPerspective(img, M_warp, (img.shape[1], img.shape[0]))
    if DEBUG_MODE:
        #Draw a red circle with zero radius and -1 for filled circle
        for src in src_udacity:
            image2 = cv.circle(img, np.int32(src), radius=0, color=(0, 0, 255), thickness=5)
        cv.imshow("Transformation points", image2)
    return image

@deprecated
def rewarp_image_udacity(img):
    img = cv.warpPerspective(img, M_rewarp, (img.shape[1], img.shape[0]), cv.WARP_INVERSE_MAP)
    return img

def rewarp_points_udacity(points):
    """
    Rewarp points from warped image coordinates to original image coordinates.
    
    Args:
        points (np.array): float32 array 
        
    @return: points in original image coordinates
    """
    return cv.perspectiveTransform(points, M_rewarp)


# "Sliding Windows"

Die Erkennung der Fahrbahnmarkierungen wird mit den sogenannten "Sliding Windows" umgesetzt. 

### <p style="color: red"> Es passiert genau das was ich dachte -> Genauer beschreiben: Histogram über ganzes Bild, Peaks von weißen Pixeln auf X-Koordinaten, Splitten des Bild in zwei Hälften -> 2 Sliding-Windows, Höhe der Windows anhand der Anzahl berechnet </p>

Kurz gefasst sind "Sliding Windows" kleine Kästchen, die anhand der Helligkeit von Pixel ihren Mittelpunkt anpassen. Damit kann beispielsweise eine weiße Linie auf schwarzem Hintergrund fast perfekt verfolgt werden. 

Detailreicher beginnt die ```sliding_windows``` Funktion damit, anhand des Histogramms eines Frames die initialen Mittelpunkte der ersten Windows festzulegen. Anhand dieser Mittelpunkte werden eine angegebene Anzahl an Windows erzeugt, die aufeinander gestapelt werden. Das bedeutet, der X-Wert der Windows bleibt gleich, der Y-Wert passt sich an.

Für jedes Window wird nun geprüft, wie viele hellen Pixel von dem jeweiligen Window eingegrenzt werden. Wenn der Wert der gefundenen Pixel das angegebene Minimum unterschreitet, wird der Mittelpunkt des jeweiligen Windows angepasst.

Ein Beispiel für diese "Sliding Windows" zeigt Abbildung 4.

<p align="center">
  <img src="img/documentation/slidingWindows.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 4: Beispiel für die Darstellung der "Sliding Windows"</em>
</p>

Damit kann die Fahrspur bei einer Veränderung leicht weiterverfolgt und markiert werden. Die hier gefundenen Mittelpunkte bilden die Grundlage für die Polynomberechnung.

Eine Alternative zu "Sliding Windows" wäre, die Spur mit Hilfe einer Kantenerkennung durchzuführen. Hierbei entsteht aber das Problem, dass selbst kleine Kanten wie die bei einem Asphaltwechsel einen enormen Störfaktor darstellen. "Sliding Windows" sind dementsprechend einfach präziser und optimieren die Erkennung deutlich, weshalb wir uns für eine Umsetzung damit entschieden haben.

In [ ]:
number_windows = 50
margin = 50
def sliding_windows(frame, window_width=200, minimum_whites=30):
    """Divides frame into multiple lines of equal height.
    For every frame the peaks of the histogram for a right and a left side are evaluated.
    One window is placed on both sides with the peak point as middle point.
    If there are more white pixels than specified with minimum_whites inside the specified boundaries of the box, its middle point is stored as as
    good value inside the ..._goods arrays. (separated for right and left windows for each line).
    If the average of the pixels are not in the center of the sliding window, the new preset center for the next sliding window is set to
    the point of highest density inside the current window.

    Args:
        frame (image): input frame with masked lane lines
        window_width (int, optional): width of windows. Defaults to 200.
        minimum_whites (int, optional): minimum of white pixel per sliding window to in order to be marked as valid. Defaults to 30.

    Returns:
        lefts, rights (np.array): returns left and right points as 2d array of sliding window centers
    """
    # Histogram for image
    hist = np.sum(frame[frame.shape[0]//2:, :], axis=0)
        
    # Take peaks from left and right side of histogram for starting points and add half margin
    mid_point_x = np.int32(hist.shape[0] // 2)
    left_x_start = np.argmax(hist[:mid_point_x]) - window_width // 2
    right_x_start = np.argmax(hist[mid_point_x:]) + mid_point_x + window_width // 2
    # Window height based on number of windows
    window_height = np.int32(frame.shape[0] // number_windows)
    
    # Calc points that are not zero in images
    nonzero = frame.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    
    # Initialize current positions for windows
    left_x_current = left_x_start
    right_x_current = right_x_start

    # Initialize values to be returned -> centers of windows
    lefts_good = np.empty((0,2), dtype=np.int32)
    rights_good = np.empty((0,2), dtype=np.int32)

    # Go through every window
    for window in range(number_windows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = frame.shape[0] - (window + 1) * window_height
        win_y_high = frame.shape[0] - window*window_height
        y_mid = (win_y_low + win_y_high) // 2
        
        # Calculate boundaries of the window
        win_xleft_low = left_x_current - window_width  
        win_xleft_high = left_x_current + window_width  
        win_xright_low =  right_x_current - window_width 
        win_xright_high = right_x_current + window_width  
        
        # Identify the pixels that are not zero within window
        left_inds = ((nonzero_y >= win_y_low ) & (nonzero_y < win_y_high) & (nonzero_x >= win_xleft_low) & (nonzero_x < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzero_y >= win_y_low ) & (nonzero_y < win_y_high) & (nonzero_x >= win_xright_low) & (nonzero_x < win_xright_high)).nonzero()[0]
        
        # If more than minimum pixels are found -> recenter next window
        if len(left_inds) > minimum_whites:
            left_x_current = np.int32(np.mean(nonzero_x[left_inds]))
            lefts_good = np.concatenate((lefts_good, [[left_x_current, y_mid]]))
            if DEBUG_MODE:
                cv.rectangle(frame, (left_x_current - margin, win_y_low),(left_x_current + margin, win_y_high),(255, 255, 255), 2)
        if len(right_inds) > minimum_whites:
            right_x_current = np.int32(np.mean(nonzero_x[right_inds]))
            rights_good = np.concatenate((rights_good, [[right_x_current, y_mid]]))
            if DEBUG_MODE:
                cv.rectangle(frame, (right_x_current - margin, win_y_low),(right_x_current + margin, win_y_high),(255, 255, 255), 2)

    return lefts_good, rights_good

# Polynome

Um eine Veränderung der Fahrspur, z.B. bei Kurven, angemessen darstellen zu können, werden anhand der von ```sliding_windows``` gefundenen Punkte Polynome berechnet.

Um zu verhindern, dass sich die Polynome durch ungültige oder zu wenig Punkte nicht ordentlich an die Fahrspur anpassen, werden die unten beschriebenen Abfragen ausgeführt.

1. Wenn die Anzahl der an die Funktion ```calculate_polynomial_points``` übergebenen Punkte pro Seite (rechts und links) kleiner ist als zwei, so werden keine Polynome berechnet, da diese sonst zu ungenau sind. Das bedeutet, es werden auch keine Punkte anhand von Polynomen berechnet, es wird also "None" als Rückgabewert festgelegt. Was das für die Spurerkennung bedeutet, wird im nächsten Abschnitt beschrieben.

2. Sind genug Punkte zur Berechnung eines Polynoms vorhanden, werden diese berechnet. Daraufhin wird geprüft, ob der Parameter des höchsten Grades der Polynome einen vordefinierten Wert überschreitet. Wenn das der Fall ist, enthalten die gegebene Punkte zur Polynomberechnung ungültige Werte. Das führt zu einer extremen, unrealistischen Krümmung der erkannten Fahrspur. Deshalb werden auch hier die berechneten Punkte nicht zur weiteren Verarbeitung zurückgegeben.

3. Sollten sich die anhand der Polynome berechneten Punkte zu nahe sein, ist mit hoher Wahrscheinlichkeit ein Schnittpunkt vorhanden. Da auch dieses Szenario in keinem der bereitgestellten Videos vorkommt, werden auch hier keine Punkte zurückgegeben. Falls die Distanz zwischen der berechneten Punkte zu groß wird, werden diese Punkte auch gestrichen. Das kann damit begründet werden, dass sich die Fahrspur in keinem der gegebene Videos plötzlich verbreitert.

Sollten die Punkte jedoch vollständig gültig sein bilden diese die rechten und linken Grenzen der einzufärbenden Fläche. Dadurch lassen sich selbst kleine Änderungend der Fahrspur ordentlich darstellen (siehe Abbildung 5).

<p align="center">
  <img src="img/documentation/polynoms.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 5: Darstellung der berechneten Polynome auf der Fahrspur</em>
</p>

Zum Einzeichnen der gesamten Fahrspur werden die anhand der berechneten Polynome ausgerechneten Punkte an die nächste Funktion ```drawRecOnFrame``` weitergegeben.

In [ ]:
#  calculate polynomial from rewarped points
max_polynom_curvature_value = 0.005 # defines how much the polynom can be curved before it is considered faulty

def calculate_polynomial_points(lefts, rights):
    """
    calculate polynomial from rewarped points and then return list of points on the polynomial
    @return: array of points for left and right polynomial
    """
    # flags for faulty polynom sides
    is_left_faulty, is_right_faulty = False, False
    
    # check if there are enough points
    if len(lefts) < 2 or len(rights) < 2:
        return None, None
    
    # calculate polynomial from rewarped points
    left_polynom_values = np.polyfit(lefts[:,1], lefts[:,0], 2)
    right_polynom_values = np.polyfit(rights[:,1], rights[:,0], 2)

    # check if polynom is valid (not too steeply curved)
    if (left_polynom_values[0] > max_polynom_curvature_value or left_polynom_values[0] < -max_polynom_curvature_value):
        is_left_faulty = True
    if (right_polynom_values[0] > max_polynom_curvature_value or right_polynom_values[0] < -max_polynom_curvature_value):
        is_right_faulty = True
    if is_left_faulty and is_right_faulty:
        return None, None
    
    # 750 as x length of polynom
    x_axis = np.linspace(0, 750, 750)

    # calculate y values for left and right line
    left_line_y = left_polynom_values[0]*x_axis**2 + left_polynom_values[1]*x_axis + left_polynom_values[2]
    right_line_y = right_polynom_values[0]*x_axis**2 + right_polynom_values[1]*x_axis + right_polynom_values[2]

    # array of points for left and right line from x and y values
    left_pts = np.array([np.transpose(np.vstack([left_line_y, x_axis]))])
    right_pts = np.array([np.transpose(np.vstack([right_line_y, x_axis]))])
    
    # loop through all points and check if they are to close or to far away from each other
    # (if they are to close -> most likely a interception point)
    for i in range(len(left_pts[0])):
        y_distance = abs(left_pts[0][i][0] - right_pts[0][i][0])
        if y_distance < 10 or y_distance > 1000:
            return None, None

    # only return non faulty polynom points
    return left_pts if not is_left_faulty else None, right_pts if not is_right_faulty else None

# Enzeichen der Fahrspur

Mit Hilfe der Funktion ```drawRecOnFrame``` wird die Fläche zwischen den Punkten, die durch die Polynome berechnet wurden, eingezeichnet (siehe Abbildung 6).

<p align="center">
  <img src="img/documentation/filledLane.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 6: Markierung der Fahrbahn mit Polynomen als Grenzen</em>
</p>

Falls, wie oben beschrieben, nicht genug Punkte vorhanden sind, um die Fläche einzuzeichnen, bleibt die zuletzt eingezeichnete Fläche einfach vorhanden. In diesem Fall beschränken wir die minimale Gesamtanzahl an Punkten, die vorhanden sein müssen um die Fläche einzuzeichnen, auf vier. Das hängt mit der im vorherigen Abschnitt beschriebenen Grenze zur Berechnung der Polynome zusammen.

Zum aktuellen Zeitpunkt existiert kein Szenario, in dem die Spur nicht genug erkannt wird, um diese Funktion auszulösen. Falls die Spurerkennung im Video "harder_challenge_video" durchgeführt wird, lässt sich diese Funktion aber gut beobachten.

In [ ]:
def drawRecOnFrame(frame, left_pts, right_pts):
    """
    Rewarp points to original image coordinates and draw rectangle on frame.
    
    input: array of points from left and right lane marking in warped image
    e.g.:
      [[ 281   39]
      [ 971  163]
      [ 958  101]]
    """
    if len(left_pts) + len(right_pts) > 3:
      borderPoints = np.concatenate((np.flip(left_pts, axis=0), right_pts))
      borderPointsRewarped = rewarp_points_udacity(np.array([borderPoints], dtype=np.float32))
      borderPointsRewarpedInt = borderPointsRewarped.astype(int)
      # draw final polygon in frame
      cv.drawContours(frame, borderPointsRewarpedInt, -1, (0,255,0), -1)

# Maskierung

Mit ```applyMasks``` werden bestimmte Farbmasken, genauer eine für Weiß und eine für Gelb, auf das Bild gelegt. Damit werden nur diese zwei Farben in den Bildern hervorgehoben bzw. dargestellt. 

Diese Funktion ist inzwischen veraltet, da die Funktion ```lane_detection``` eine verbesserte Implementierung mit neuen Filtern zur Verfügung stellt.

In der Funktion ```lane_detection``` werden zwar weiterhin Farbfilter angewandt, um nur die für die Spurerkennung wichtigen Farben hervorzuheben, dazu werden aber auch noch morphologische Filter auf den Frame im HLS-Farbraum angewandt.
Dazu gehört zum Beispiel "Opening", um einzelne Teile der Spur von anderen zu trennen und somit leichter erkennbar zu machen. 

Zusammengefasst lässt sich sagen, dass mit Hilfe der Funktion ```lane_detection``` für jede relevante Farbe (Weiß und Gelb) eine verbesserte Hervorhebung durch Masken und Filter entsteht. Dadurch können die Fahrbahnmarkierungen mit Hilfe der Sliding-Windows deutlich robuster verfolgen bzw. erkennen, was auch der Grund für einen Wechsel zur neuen Funktion darstellt.

### <p style="color: red"> Genauer? </p>

In [ ]:
@deprecated
def applyMasks(frame):
    """
    Apply masks to frame and return frame with only lane marking.
    """
    ## convert to hsv
    hls_frame = cv.cvtColor(frame, cv.COLOR_BGR2HLS)

    ## mask for white
    white_mask = cv.inRange(hls_frame, (0, 200, 0), (255, 255,255))

    ## mask for yellow
    # yellow_mask = cv.inRange(hls_frame, (20,90,200), (26, 255, 255))
    yellow_mask = cv.inRange(hls_frame, (10,0,100), (40, 255, 255))

    ## final mask and masked
    mask = cv.bitwise_or(white_mask, yellow_mask)
    frame = cv.bitwise_and(frame,frame, mask=mask)

    return frame

In [ ]:
def lane_detection(frame, white_lower, white_upper, yellow_lower, yellow_upper):
    """ Use color masks to detect lane markings in bird's eye view. 
    
    Uses Gaussian blur and HSV color space, then applies masks for white and yellow lane markings. Applies morphological operations to remove noise and fill gaps. Then combines masks and applies them to the input frame.

    Args:
        frame (_type_): input frame to mask lane markings in bird's eye view
        white_lower (_type_): lower bound for white mask
        white_upper (_type_): upper bound for white mask
        yellow_lower (_type_): lower bound for yellow mask
        yellow_upper (_type_): upper bound for yellow mask

    Returns:
        image: masked image with lane markings
    """
    if DEBUG_MODE: # needs frame at this point
        greenImg = np.zeros(frame.shape, frame.dtype)
        redImg = np.zeros(frame.shape, frame.dtype)
        
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    frame_hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS) #-> Weiß
    frame_lab = cv.cvtColor(frame, cv.COLOR_BGR2LAB) # -> Gelb

    white_mask = cv.inRange(frame_hls, white_lower, white_upper)
    # white_mask[:, 0:200] = 0 # could be used for harder challenge video to ignore left side of image for white mask

    yellow_mask = cv.inRange(frame_lab, yellow_lower, yellow_upper)
    # yellow_mask[:, 1000:] = 0 # could be used for harder challenge video to ignore right side of image for yellow mask

    combined_mask = cv.bitwise_or(white_mask, yellow_mask)
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (13, 13))
    lanes_yellow = cv.morphologyEx(frame_lab[:, :, 2], cv.MORPH_TOPHAT, kernel) # -> kleine helle sachen in dunkler umgebung hervorheben (hebt Gelb deutlich hervor im LAB-Farbraum)
    ret, lanes_yellow = cv.threshold(lanes_yellow, thresh=2, maxval=255, type=cv.THRESH_BINARY) 
    lanes_yellow = cv.morphologyEx(lanes_yellow, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=4)

    combined_mask = cv.bitwise_or(combined_mask, lanes_yellow) # -> Lanes_yellow = verbesserter Yellowfilter
    combined_mask = cv.morphologyEx(combined_mask, cv.MORPH_OPEN, np.array([[0,1,0],[1,1,1],[0,1,0]], 'uint8'), iterations=2)

    frame = cv.bitwise_and(frame, frame, mask=combined_mask)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    if DEBUG_MODE: # show combined debug images
        redImg[:,:] = (0, 0, 255)
        redMask = cv.bitwise_and(redImg, redImg, mask=yellow_mask)

        greenImg[:,:] = (0, 255, 0)
        greenMask = cv.bitwise_and(greenImg, greenImg, mask=white_mask)

        frame_mask = redMask + greenMask

        cv.imshow("Current Mask", frame_mask)
        cv.imshow("Combined Mask", combined_mask)


    return frame

# Helligkeitskorrektur

Damit die Fahrbahnmarkierungen in dunklen sowie in sehr hellen Elementen der Videos zu erkennen sind, wurde eine dynamische Helligkeitsanpassung als Zusatzfunktion implementiert.

Der Frame wird in den HSV-Farbraum umgewandelt und in die jeweiligen Bestandteile aufgeteilt. Daraufhin wird der durchschnittliche Helligkeitswert des Frames berechnet.

Wenn dieser Helligkeitswert dann kleiner oder größer als die angegebenen Grenzen ist, so wird die Helligkeit des Frames angepasst.

Bei sehr hellen Bereichen wird die durchschnittliche Hellligkeit soweit nach unten korrigiert, dass sie sich dann zwischen den gesetzten Grenzen befindet (vgl. Abbildung 7). 
Selbes gilt für dunkle Bereiche.

<p align="middle" float="left">
  <img src="img/documentation/brightnessCorrectionNormal.png" width="40%" height="40%"/>
  <img src="img/documentation/brightnessCorrectionCorrected.png" width="40%" height="40%"/>
  <br>
  <em>Abbildung 7: Helligkeitskorrektur - Links: Original, Rechts: Korrigiert</em>
</p>

Damit werden Teile der Fahrbahnmarkierungen, die davor nicht von den festgelegten Masken und Filter erkannt werden konnten, so angepasst, dass sie im gefilterten Bild zu erkennen sind. Das macht die Fahrspurerkennung deutlich robuster.

In [ ]:
def correct_Brightness(image):
    """Calculate the brightness of the overall image and adjust it if it is too bright or too dark.

    Args:
        image: input image

    Returns:
        image, brightness: corrected image and new overall brightness
    """
    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image_brightness_input = image_hsv[...,2].mean()
    brightness_upper_limit = 150
    brightness_lower_limit = 85
    if image_brightness_input > brightness_upper_limit:
        h, s, v = cv.split(image_hsv)
        
        lim = round(0 + image_brightness_input - brightness_upper_limit)
        v[v < lim] = 0
        v[v >= lim] -= lim

        final_hsv = cv.merge((h, s, v))
        image = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    
    elif image_brightness_input < brightness_lower_limit:
        h, s, v = cv.split(image_hsv)
        
        lim = round(255 + brightness_lower_limit - image_brightness_input)
        v[v > lim] = 255
        v[v <= lim] += lim

        final_hsv = cv.merge((h, s, v))
        image = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)

    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image_brightness_new = image_hsv[...,2].mean()

    return image, image_brightness_new

In [ ]:
def correct_Brightness_and_Split(image):
    # ----------- Split in quadrants ---------------
    start_time_measurement("quadrant splitting")
    height_cutoff = image.shape[0] // 2
    width_cutoff = image.shape[1] // 2

    left_side = image[:, :width_cutoff]
    right_side = image[:, width_cutoff:]

    l1 = left_side[:height_cutoff, :]
    l2 = left_side[height_cutoff:, :]

    r1 = right_side[:height_cutoff, :]
    r2 = right_side[height_cutoff:, :]
    end_time_measurement("quadrant splitting")
    
    # ----------- Correct brightness ---------------
    start_time_measurement("brightness correction")
    l2, brightness_l2 = correct_Brightness(l2)
    l1, brightness_l1 = correct_Brightness(l1)
    r2, brightness_r2 = correct_Brightness(r2)
    r1, brightness_r1 = correct_Brightness(r1)
    end_time_measurement("brightness correction")
    
    # ----------- Merge the quadrants ---------------
    start_time_measurement("quadrant merging")
    numpy_vertical_l = np.vstack((l1, l2))
    numpy_vertical_r = np.vstack((r1, r2))
    grayscale_frame = np.hstack((numpy_vertical_l, numpy_vertical_r))
    end_time_measurement("quadrant merging")
    
    if DEBUG_MODE:
        # draw borders around the quadrants and display the brightness
        l2_border = cv.copyMakeBorder(l2, 2, 0, 0, 2, cv.BORDER_CONSTANT, value=(255, 0, 0))
        l1_border = cv.copyMakeBorder(l1, 0, 2, 0, 2, cv.BORDER_CONSTANT, value=(255, 0, 0))
        r2_border = cv.copyMakeBorder(r2, 2, 0, 2, 0, cv.BORDER_CONSTANT, value=(255, 0, 0))
        r1_border = cv.copyMakeBorder(r1, 0, 2, 2, 0, cv.BORDER_CONSTANT, value=(255, 0, 0))

        cv.putText(l2_border, "Brightness: " + str(np.round(brightness_l2)), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(l1_border, "Brightness: " + str(np.round(brightness_l1)), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(r2_border, "Brightness: " + str(np.round(brightness_r2)), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        cv.putText(r1_border, "Brightness: " + str(np.round(brightness_r1)), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)

        numpy_vertical_l_brightness = np.vstack((l1_border, l2_border))
        numpy_vertical_r_brightness = np.vstack((r1_border, r2_border))
        grayscale_frame_brightness = np.hstack((numpy_vertical_l_brightness, numpy_vertical_r_brightness))
        cv.imshow("Current Brightness", grayscale_frame_brightness)

    return grayscale_frame    

# Debug-Modus - Slider

Diese Slider stellen eine weitere Zusatzfunktion unserer Implementierung dar. Wenn der Debug-Modus aktiviert ist, können mit Hilfe dieser Slider Einstellungen, wie z.B. die Werte der Farbmasken für jede Farbe, verändert werden.

Damit ließen sich die Werte der jeweiligen Masken an einem Live-Bild leichter optimieren.

In [ ]:
def on_change(value):
    """
    Callback function that gets called whenever one of the slider inside the Options window is changed.
    Global variable next_frame is set to true and therefore onscreen images get updated.
    """
    global next_frame
    next_frame = True
    
def init_sliders():
    """Creates option window with sliders for mask range and frame selection."""
    cv.namedWindow("Options", cv.WINDOW_NORMAL)
    
    global frames
    global white_lower
    global white_upper
    global yellow_lower
    global yellow_upper

    cv.createTrackbar('Autoplay', 'Options', 0, 1, on_change)
    
    cv.createTrackbar('Frame', 'Options', 0, len(frames), on_change)

    cv.createTrackbar('w_low1', 'Options', 0, 255, on_change)
    cv.createTrackbar('w_low2', 'Options', 0, 255, on_change)
    cv.createTrackbar('w_low3', 'Options', 0, 255, on_change)

    cv.createTrackbar('w_up1', 'Options', 0, 255, on_change)
    cv.createTrackbar('w_up2', 'Options', 0, 255, on_change)
    cv.createTrackbar('w_up3', 'Options', 0, 255, on_change)

    cv.createTrackbar('y_low1', 'Options', 0, 255, on_change)
    cv.createTrackbar('y_low2', 'Options', 0, 255, on_change)
    cv.createTrackbar('y_low3', 'Options', 0, 255, on_change)

    cv.createTrackbar('y_up1', 'Options', 0, 255, on_change)
    cv.createTrackbar('y_up2', 'Options', 0, 255, on_change)
    cv.createTrackbar('y_up3', 'Options', 0, 255, on_change)


    cv.setTrackbarPos('w_low1', 'Options', white_lower[0])
    cv.setTrackbarPos('w_low2', 'Options', white_lower[1])
    cv.setTrackbarPos('w_low3', 'Options', white_lower[2])

    cv.setTrackbarPos('w_up1', 'Options', white_upper[0]) 
    cv.setTrackbarPos('w_up2', 'Options', white_upper[1])
    cv.setTrackbarPos('w_up3', 'Options', white_upper[2])

    cv.setTrackbarPos('y_low1', 'Options', yellow_lower[0])
    cv.setTrackbarPos('y_low2', 'Options', yellow_lower[1])
    cv.setTrackbarPos('y_low3', 'Options', yellow_lower[2])

    cv.setTrackbarPos('y_up1', 'Options', yellow_upper[0])
    cv.setTrackbarPos('y_up2', 'Options', yellow_upper[1])
    cv.setTrackbarPos('y_up3', 'Options', yellow_upper[2])


# Main

Folgender Abschnitt zeigt den gesamten Ablauf der Fahrspurerkennung.

In Zeile 2 lässt sich festlegen, in welchem der gegebenen Videos die Spurerkennung durchgeführt werden soll. Zur Auswahl stehen: "project", "challenge" und "harder_challenge".

Nach einem Durchlauf des Programms findet sich am Ende des Notebooks die oben angesprochene Tabelle, die den zeitlichen Anteil der einzelnen Funktionen zeigt.

Zusätzlich zu den oben beschriebenen Slidern wurde die im Debug-Modus aktive Funktion implementiert, mit der Taste "w" einen Frame weiter und mit "s" einen Frame zurück zu springen. Auch hier lieferte diese Funktion den elementaren Vorteil, Fehler bei der Spurerkennung besser zu lokalisieren und Optimierungen zu finden.

<p style="color: red"> ------------------------------------------------------------ Evtl. verschieben ------------------------------------------------------------ </p>

Um die Erkennung der Fahrspur noch weiter zu verbessern, wurde das Frame in vier Quadranten eingeteilt. Diese Quadranten werden als jeweils einzelne Frames betrachtet. Somit kann die Helligkeitsanpassung schneller auf Helligkeitsschwankungen ragieren. Damit können, beispielsweise im "challenge"-Video unter der Brücke, vorherige Fahrbahnmarkierungen länger und folgenden Fahrbahnmarkierungen früher erkannt werden.

<p style="color: red"> ------------------------------------------------------------ Evtl. verschieben ------------------------------------------------------------ </p>
Um zu demonstrieren, dass unsere Optimierung mit Hilfe der Helligkeitsanpassung und dem Aufteilen des Frames in vier Quadranten eine wirkliche Optimierung darstellt, wurden beide Videos ("project_video" und "challenge_video") einmal mit und einmal ohne Helligkeitsanpassung durchlaufen und abgespeichert. Im aktuellen Code ist die Helligkeitserkennung jedoch standardmäßig aktiviert.


In [ ]:
# Open video file
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# variables for frame selection
next_frame = True
position = 0

# Read all frames. This is necessary to be able to use the slider to select a frame
frames = []
while capture.isOpened():
    ret, image = capture.read()

    # Check if there is another frame
    if image is None:
        break

    frames.append(image)
# Release video capture
capture.release()  

#play like a video in debug mode
autoplay = False

# define mask ranges
yellow_lower = np.array([150, 100, 140])
yellow_upper = np.array([255, 140, 200])

white_lower = np.array([0, 180, 0])
white_upper = np.array([254, 254, 254])

if DEBUG_MODE: # enable sliders to change mask ranges in real time 
    init_sliders()
    autplay = False

# cache between frames
old_left_pts = None
old_right_pts = None

# reset time analysis
time_measurements = {}

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

# Read every frame
while position < len(frames):
    image = frames[position]

    # Check if there is another frame
    if image is None:
        break

    if next_frame:

        if DEBUG_MODE: # update sliders
            white_lower[0] = cv.getTrackbarPos('w_low1', 'Options')
            white_lower[1] = cv.getTrackbarPos('w_low2', 'Options')
            white_lower[2] = cv.getTrackbarPos('w_low3', 'Options')

            white_upper[0] = cv.getTrackbarPos('w_up1', 'Options')
            white_upper[1] = cv.getTrackbarPos('w_up2', 'Options')
            white_upper[2] = cv.getTrackbarPos('w_up3', 'Options')

            yellow_lower[0] = cv.getTrackbarPos('y_low1', 'Options')
            yellow_lower[1] = cv.getTrackbarPos('y_low2', 'Options')
            yellow_lower[2] = cv.getTrackbarPos('y_low3', 'Options')

            yellow_upper[0] = cv.getTrackbarPos('y_up1', 'Options')
            yellow_upper[1] = cv.getTrackbarPos('y_up2', 'Options')
            yellow_upper[2] = cv.getTrackbarPos('y_up3', 'Options')

        # Calculate Frame rate
        frame_count += 1
        elapsed_time = time.time() - start_timer
        frame_rate = frame_count / elapsed_time
        
        start_time_measurement("frame")
        
        # --------- Preprocessing ---------
        start_time_measurement("Preprocessing")
        image = undistort_image_remap(image)
        frame_undistorted = image.copy()
        image = warp_image_udacity(image)
        end_time_measurement("Preprocessing")

        # ---------  Brightness correction ---------
        image = correct_Brightness_and_Split(image)

        # ---------  Masking ---------
        start_time_measurement("masking")
        frame_mask = lane_detection(image, white_lower, white_upper, yellow_lower, yellow_upper)
        end_time_measurement("masking")

        # --------- Sliding Windows ---------
        start_time_measurement("sliding windows")
        lefts, rights = sliding_windows(frame_mask, minimum_whites=margin)
        end_time_measurement("sliding windows")

        # --------- Calculate polynomial ---------
        start_time_measurement("polynomial calculation")
        left_pts, right_pts = calculate_polynomial_points(lefts, rights)
        end_time_measurement("polynomial calculation")

        # --------- draw plane from all sliding windows ---------
        start_time_measurement("draw plane")
        if left_pts is not None:
            old_left_pts = left_pts        
        if right_pts is not None:
            old_right_pts = right_pts
        drawRecOnFrame(frame_undistorted, old_left_pts[0], old_right_pts[0])
        if old_left_pts is  None and old_right_pts is  None:
            print("No plane drawn")
        end_time_measurement("draw plane")

        # --------- Add frame rate to video ---------
        if not DEBUG_MODE:
            cv.putText(frame_undistorted, "FPS: " + str(round(frame_rate)), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
            cv.putText(frame_undistorted, "Frame: " + str(position), (0, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)
        else:
            cv.putText(frame_undistorted, "Frame: " + str(position), (0, 25), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv.LINE_AA,)

        cv.imshow("Frame", frame_undistorted)

        end_time_measurement("frame")    
        
    pressedKey = cv.waitKey(1) & 0xFF
    if pressedKey == ord('q'): # press 'q' to exit the video
        break
    
    if autoplay or not DEBUG_MODE:
        position += 1
        next_frame = True
    
    if DEBUG_MODE: # DEBUG_MODE enables the user to navigate through the video with the keyboard ('w', 's') or the slider in the window
        if cv.getTrackbarPos('Autoplay', 'Options') != int(autoplay):
            autoplay = bool(cv.getTrackbarPos('Autoplay', 'Options'))

        if autoplay:
            cv.setTrackbarPos('Frame', 'Options', position)

        if cv.getTrackbarPos('Frame', 'Options') != position:
            position = cv.getTrackbarPos('Frame', 'Options')

        if pressedKey == ord('w'):
            position += 1
            next_frame = True
            cv.setTrackbarPos('Frame', 'Options', position)
        elif pressedKey == ord('s') and position > 0:
            position -= 1
            next_frame = True
            cv.setTrackbarPos('Frame', 'Options', position)
        elif pressedKey == 32:
            autoplay = not autoplay
            cv.setTrackbarPos('Autoplay', 'Options', int(autoplay))

# When everything done, close all windows
cv.destroyAllWindows()

# create time measurement analysis and print out results
analyse_time_measurements()

# Fazit

### <p style="color: red"> Todo zusammen </p>

- Perspektivtransformation erleichtert Spurerkennung enorm
- Man kann sich leicht in Verbesserungen verrennen
- Harder_challenge braucht mehr Verbesserung/Anpassung als gedacht
- Sliding Windows geben beste Genauigkeit, Edge-Filter erkennen auch Asphaltwechsel -> erkennen zu viel, egal wie viel man mit Farb- und Morphologischen Filtern arbeitet
- Reine Infos aus der Vorlesung reichen definitv nicht aus, um die Spurerkennung effektiv umzusetzen
- Eigene Funktionen wie eine Zeitmessung sind essenziel für Optimierung an der richtigen Stelle


## Ausblick
- Multiprocessing der Helligkeitskorrektur und masking für einzele Quadranten